In [1]:
# Import Libraries

import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot

## Pandas Missing Extension

In [ ]:
%run pandas-missing-extension.ipynb

In [ ]:
# Settings
%matplotlib inline

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',200)
pd.options.display.float_format= '{:.2f}'.format

# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

sns.set(
    rc={
        "figure.figsize": (10, 10)
    }
)

sns.set_style("whitegrid")

### Extract Data

In [ ]:
url = "https://github.com/njtierney/naniar/raw/master/data/"

!wget -O ./data/pedestrian.csv { url } -q